<a href="https://colab.research.google.com/github/AymenTlili131/The-sound-of-Ai-Hack-V1.0/blob/working_proposals/MIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
from os import walk
import librosa.display
from natsort import natsorted
import pickle

In [2]:
from google.colab import drive
###working with a Drive version of the dataset
drive.mount('/content/drive',force_remount= True)

Mounted at /content/drive


In [4]:
def extract_features(x, feature, win_length,n_mfcc, sr, n_mels):
    #MIR extraction 
    hop_length = int(win_length/2)
    rms = librosa.feature.rms(x, frame_length= win_length, hop_length=hop_length, center=True)[0]
    spec_cent = librosa.feature.spectral_centroid(y= x,sr= sr,hop_length= hop_length,win_length= win_length)[0]  
    zcr = librosa.feature.zero_crossing_rate(x, frame_length= win_length, hop_length= hop_length)[0]
    #D = np.abs(librosa.stft(x))**2
    #mel = librosa.feature.melspectrogram(S=D, sr=sr) 
    mel = librosa.feature.melspectrogram(x, sr= sr, hop_length= hop_length,n_mels= n_mels)
    mfcc = librosa.feature.mfcc(x, sr= sr,n_mfcc= n_mfcc)
    duration = x.shape[0]/sr
    bpm = librosa.beat.tempo(x, sr= sr)[0]

    if feature == "rms":
        return rms
    if feature == "mfcc": 
        return mfcc
    if feature == "mel": 
        return mel
    if feature == "sc": 
        return spec_cent
    if feature == "zcr":
        return zcr
    if feature == "duration":
        return duration
    if feature == "bpm":
        return bpm


In [ ]:
#####################
#####################
#####################
#####################
#####################
#####################
#####################

In [ ]:
artist= os.listdir('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/')
print(artist)

['Sweden', 'Mexico', 'USA', 'India']


In [ ]:
from pathlib import Path
dirs_wavs = sorted(Path('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/Sweden').glob('**/*.wav'))
# print(len(dirs_wavs))
artists = [str(x).split('/')[-3] for x in dirs_wavs]
songs = [str(x).split('/')[-1] for x in dirs_wavs]
audios = [str(dirs_wavs[idx]) for idx, dir in enumerate(dirs_wavs)]
for i in range(len(dirs_wavs)):
    print(f'Artist {i}',artists[i])
    print(f'Song {i}', songs[i])
    print(f'Audio {i}', audios[i])
    print('\n')
assert len(dirs_wavs) == len(artists) == len(songs) == len(audios)

In [43]:
audio_files = []
for audio in audios:
    #print(audio)
    x, fs = librosa.load(audio, sr= 44100)
    audio_files.append(x)
audio_files = np.array(audio_files)
print(audio_files.shape)


(49,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [44]:
import IPython.display as ipd
ipd.Audio(audio_files[0][:44100],rate=44100)


In [45]:
MEL = []
hop_length= 1024
DURATION = 10
SR = 44100
for i in audio_files:
        MEL.append(extract_features(i[:SR * DURATION],'mel',2048, n_mfcc= 13, sr= SR,n_mels = 128))
#MEL = np.array(MEL)
#print(MEL.shape)

In [46]:
RMS = []
hop_length= 1024
DURATION = 10
SR = 44100
for i in audio_files:
        RMS.append(extract_features(i[:SR * DURATION],'rms',2048, n_mfcc= 13, sr= SR,n_mels = 128))
RMS = np.array(RMS)
print(RMS.shape)

(49, 431)


In [47]:
SC = []
hop_length= 1024
DURATION = 10
SR = 44100
for i in audio_files:
        SC.append(extract_features(i[:SR * DURATION],'sc',2048, n_mfcc= 13, sr= SR,n_mels = 128))
SC= np.array(SC)
print(SC.shape)

(49, 431)


In [48]:
ZCR = []
hop_length= 1024
DURATION = 10
SR = 44100
for i in audio_files:
        ZCR.append(extract_features(i[:SR * DURATION],'zcr',2048, n_mfcc= 13, sr= SR,n_mels = 128))
ZCR = np.array(ZCR)
print(ZCR.shape)

(49, 431)


In [49]:
DUR = []
hop_length= 1024
DURATION = 10
SR = 44100
for i in audio_files:
        DUR.append(extract_features(i[:SR * DURATION],'duration',2048, n_mfcc= 13, sr= SR,n_mels = 128))
DUR = np.array(DUR)
print(DUR.shape)

(49,)


In [50]:
BPM = []
hop_length= 1024
DURATION = 10
SR = 44100
for i in audio_files:
        BPM.append(extract_features(i[:SR * DURATION],'bpm',2048, n_mfcc= 13, sr= SR,n_mels = 128))
BPM = np.array(BPM)
print(BPM.shape)

(49,)


In [ ]:
import pandas as pd

my_dict = {
    "artist": artists,
    "song": songs,
    "mel": MEL,
    "rms(mean)": np.mean(RMS,axis = 1 ),
           "rms(var)": np.var(RMS, axis = 1 ),
           "sc(mean)": np.mean(SC,axis = 1),
           "sc(var)": np.var(SC,axis = 1),
           "zcr(mean)": np.mean(ZCR, axis = 1),
          "zcr(var)": np.mean(ZCR, axis = 1),
           "duration": DUR,
           "bpm": BPM,
          }

audio_features = pd.DataFrame(my_dict)
display(audio_features)

In [53]:
from google.colab import files

audio_features.to_csv('MIR_Sweden.csv', encoding = 'utf-8-sig') 
files.download('MIR_Sweden.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>